**Initialize database**

In [162]:

/* Drop Tables */

set role postgres;

DROP TABLE IF EXISTS accidents;
DROP TABLE IF EXISTS archive;
DROP TABLE IF EXISTS manager_on_contract;
DROP TABLE IF EXISTS raw_contracts;
DROP TABLE IF EXISTS cars;
DROP TABLE IF EXISTS clients;
DROP TABLE IF EXISTS managers;
DROP TABLE IF EXISTS parkings;
DROP TABLE IF EXISTS users;




/* Create Tables */

CREATE TABLE accidents
(
	ID_accident int NOT NULL UNIQUE,
	is_confirmed boolean NOT NULL,
	driving_license varchar(10) NOT NULL,
	vin_number varchar(100) NOT NULL UNIQUE,
	PRIMARY KEY (ID_accident)
) WITHOUT OIDS;


CREATE TABLE archive
(
	ID_record serial UNIQUE,

	ID_contract int NOT NULL,
	start_date date NOT NULL,
	end_date date NOT NULL,
	total_price money NOT NULL,

	car_vin_number varchar(100) NOT NULL UNIQUE,
	
	client_driving_license varchar(10) NOT NULL UNIQUE,
	client_name varchar(100) NOT NULL,
	phone_number varchar(20) UNIQUE,

	manager_passport_details varchar(90) NOT NULL UNIQUE,
	manager_name varchar(100) NOT NULL,

	PRIMARY KEY (ID_record)
) WITHOUT OIDS;


CREATE TABLE cars
(
	vin_number varchar(100) NOT NULL UNIQUE,
	cost_per_hour money,
	car_brand varchar(40) NOT NULL,
	car_color varchar(20) NOT NULL,
	car_class varchar(20) NOT NULL,
	transmission_type varchar(4) NOT NULL,
	fuel_type varchar(20) NOT NULL,
	locked boolean DEFAULT 'FALSE' NOT NULL,
	ID_parking int NOT NULL,
	PRIMARY KEY (vin_number)
) WITHOUT OIDS;


CREATE TABLE clients
(
	driving_license varchar(10) NOT NULL,
	name varchar(100) NOT NULL,
	phone varchar(20) UNIQUE,
	email varchar(50) UNIQUE,
	date_of_birth date,
	registration_address varchar,
	residence_address varchar,
	username varchar(20) NOT NULL UNIQUE,
	PRIMARY KEY (driving_license)
) WITHOUT OIDS;


CREATE TABLE managers
(
	manager_passport_details varchar(90) NOT NULL UNIQUE,
	name varchar(100) NOT NULL,
	phone varchar(20) NOT NULL UNIQUE,
	expirience smallint DEFAULT 0,
	salary int,
	username varchar(20) NOT NULL UNIQUE,
	ID_parking int NOT NULL UNIQUE,
	PRIMARY KEY (manager_passport_details)
) WITHOUT OIDS;


CREATE TABLE manager_on_contract
(
	manager_passport_details varchar(90) NOT NULL UNIQUE,
	ID_contract int NOT NULL UNIQUE
) WITHOUT OIDS;


CREATE TABLE parkings
(
	ID_parking int NOT NULL UNIQUE,
	address varchar(100) NOT NULL,
	PRIMARY KEY (ID_parking)
) WITHOUT OIDS;


CREATE TABLE raw_contracts
(
	ID_contract int NOT NULL UNIQUE,
	rental_duration interval NOT NULL,
	total_price money NOT NULL,
	driving_license varchar(10) NOT NULL UNIQUE,
	vin_number varchar(100) NOT NULL UNIQUE,
	signed boolean DEFAULT 'FALSE' NOT NULL,
	PRIMARY KEY (ID_contract)
) WITHOUT OIDS;


CREATE TABLE users
(
	username varchar(20) NOT NULL UNIQUE,
	password varchar(30) NOT NULL,
	role varchar(10) NOT NULL,
	PRIMARY KEY (username)
) WITHOUT OIDS;



/* Create Foreign Keys */

ALTER TABLE accidents
	ADD FOREIGN KEY (vin_number)
	REFERENCES cars (vin_number)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE accidents
	ADD FOREIGN KEY (driving_license)
	REFERENCES clients (driving_license)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE raw_contracts
	ADD FOREIGN KEY (vin_number)
	REFERENCES cars (vin_number)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE raw_contracts
	ADD FOREIGN KEY (driving_license)
	REFERENCES clients (driving_license)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE manager_on_contract
	ADD FOREIGN KEY (manager_passport_details)
	REFERENCES managers (manager_passport_details)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE cars
	ADD FOREIGN KEY (ID_parking)
	REFERENCES parkings (ID_parking)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE managers
	ADD FOREIGN KEY (ID_parking)
	REFERENCES parkings (ID_parking)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE manager_on_contract
	ADD FOREIGN KEY (ID_contract)
	REFERENCES raw_contracts (ID_contract)
	ON UPDATE CASCADE
	ON DELETE CASCADE
;


ALTER TABLE clients
	ADD FOREIGN KEY (username)
	REFERENCES users (username)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;


ALTER TABLE managers
	ADD FOREIGN KEY (username)
	REFERENCES users (username)
	ON UPDATE NO ACTION
	ON DELETE NO ACTION
;



/* Comments */

COMMENT ON COLUMN clients.name IS 'Фамилия Имя Отчество клиента';
COMMENT ON COLUMN clients.phone IS 'Потребуется для быстрой связи с клиентом. Тип данных - могло быть и число, но для простоты учебного примера - пусть будет строка';
COMMENT ON COLUMN clients.email IS 'Электронный адрес';

SET

DROP TABLE

DROP TABLE

DROP TABLE

DROP TABLE

...

ALTER TABLE

COMMENT

COMMENT

COMMENT

Total execution time: 00:00:00.353

**Types**

In [163]:
-- drop type if exists type_transmission;

-- CREATE TYPE type_transmission AS ENUM ('A', 'M');

Total execution time: 00:00:00

**Alter table**

In [164]:
alter table cars
alter column transmission_type type type_transmission using transmission_type::type_transmission; 

ALTER TABLE

Total execution time: 00:00:00.014

**Functions**

In [165]:
drop function if exists check_auth;
drop function if exists archive_contract;
drop function if exists add_new_user_role;
drop function if exists modify_on_insert;
drop function if exists get_cars;
drop function if exists prepare_contract;
drop function if exists add_accident;
drop function if exists create_user_role;


create function check_auth(input_username varchar, input_password varchar)
returns varchar as 
$$
declare 
    user_role varchar;
begin
    select role into user_role from users where username = input_username and password = input_password;
    return user_role;
end
$$ LANGUAGE plpgsql;


create function get_cars() 
returns table (hour_cost money, class varchar, brand varchar, color varchar, transmission type_transmission, fuel varchar) as $$
begin 
    return query 
        select cost_per_hour, car_class, car_brand, car_color, transmission_type, fuel_type 
        from cars 
        where locked = false;
end;
$$ language plpgsql;



-- "Procedure-like" functions
create function prepare_contract(user_name varchar, vin varchar, duration interval)
returns varchar as $$
declare
    id int;
    hour_cost money;
    license varchar;
begin
    select floor(random() * 10000) into id;
    select cost_per_hour into hour_cost from cars where cars.vin_number = vin;
    select driving_license into license from clients where username = user_name;

    insert into raw_contracts values (id, duration, (24 * hour_cost * extract(day from duration)), license, vin);
    return id;
end;
$$ language plpgsql;

create function add_accident(user_name varchar, vin varchar)
returns varchar as $$
declare
    id int;
    license varchar;
begin
    select floor(random() * 10000) into id;
    select driving_license into license from clients where user_name = username;
    insert into accidents values (id, false, license, vin);
    return id;
end;
$$ language plpgsql;



-- Trigger functions
create function archive_contract() returns trigger as $$
declare
    client_name varchar;
    phone_number varchar;
    manager_passport varchar;
    manager_name varchar;
begin
    if new.signed = true then
        select name into client_name from clients where c.driving_license = new.driving_license;
        select phone into phone_number from clients where clients.driving_license = new.driving_license;
        select manager_passport_details into manager_passport from manager_on_contract where manager_on_contract.ID_contract = new.ID_contract;
        select name into manager_name from managers where managers.manager_passport_details = manager_passport;

        insert into archive 
            (ID_contract, start_date, end_date, total_price, car_vin_number, client_driving_license, client_name, phone_number, manager_passport_details, manager_name) 
        values 
            (new.ID_contract, now(), now() + new.rental_duration, new.total_price, new.vin_number, new.driving_license, client_name, phone_number, manager_passport, manager_name);

        delete from raw_contracts where new.ID_contract = old.ID_contract;
        return new;
    end if;
end;
$$ language plpgsql;

create function create_user_role() returns trigger as $$
begin
    if not exists (select rolname from pg_roles where new.username = rolname) then
        execute 'create user "'||new.username||'" with login connection limit 1;';
        execute 'grant _client to "'||new.username||'"';
    end if;
    return new;
end;
$$ language plpgsql;

create function modify_on_insert() returns trigger as $$
begin
    update cars set locked = true where cars.vin_number = new.vin_number;
    return new;
end;
$$ language plpgsql;

DROP FUNCTION

NOTICE: function archive_contract() does not exist, skipping

DROP FUNCTION

NOTICE: function add_new_user_role() does not exist, skipping

DROP FUNCTION

...

CREATE FUNCTION

CREATE FUNCTION

CREATE FUNCTION

CREATE FUNCTION

Total execution time: 00:00:00.050

**Test**

In [166]:
/*
select 'user1' as user, check_auth('user1', 'password1') as user_role;
select 'user3' as user, check_auth('user3', 'password3') as user_role;
select 'new user' as user, check_auth('new user', 'incorrect_password') as user_role;
*/

Total execution time: 00:00:00

**Procedures**

In [167]:
drop procedure if exists add_user;
drop procedure if exists confirm_accident;
drop procedure if exists confirm_contract;

create procedure add_user (user_name varchar, user_password varchar) as
$$
begin
    if not exists (select username from users where user_name = username) then
        insert into users values (user_name, user_password, 'client');
    end if;
end;
$$ language plpgsql;


create procedure confirm_contract (id int) as 
$$
begin 
    update raw_contracts set signed = true where id = ID_contract;
end;
$$ language plpgsql;


create procedure confirm_accident (id int, manager_username varchar) as 
$$
begin 
    insert into managers_on_contract values (
        (select manager_passport_details from managers where username = manager_username),
        id
    );
    update accidents set is_confirmed = true where id = ID_accident;
end;
$$ language plpgsql;

DROP PROCEDURE

DROP PROCEDURE

DROP PROCEDURE

CREATE PROCEDURE

CREATE PROCEDURE

CREATE PROCEDURE

Total execution time: 00:00:00.026

**Triggres**

In [168]:
CREATE OR REPLACE TRIGGER contract_signed 
    AFTER UPDATE OF signed 
    ON raw_contracts
    FOR EACH ROW
    EXECUTE PROCEDURE archive_contract();

create or replace trigger on_insert_raw_contracts
    after insert on raw_contracts
    for each row 
    execute procedure modify_on_insert();

create or replace trigger user_added
    after insert on users
    for each row
    execute procedure create_user_role();

CREATE TRIGGER

CREATE TRIGGER

CREATE TRIGGER

Total execution time: 00:00:00.012

**Test**

In [169]:
/*
delete from raw_contracts;
delete from archive;
delete from cars;
delete from parkings;
delete from clients;
delete from managers;
delete from users;

insert into parkings values 
(1, 'asfiaslfdgashdfguiapshg'),
(2, 'gflsdjiopsjdfoigaoijbdi');

INSERT INTO users (username, password) VALUES 
('user1', 'password1'),
('user2', 'password2'),
('user3', 'password3'),
('user4', 'password4');

insert into managers (manager_passport_details, name, phone, expirience, salary, username, ID_parking) values 
('manager passport 1', 'manager name 1', 'manager phone 1', DEFAULT, 10, 'user2', 1),
('manager passport 2', 'manager name 2', 'manager phone 2', DEFAULT, 20, 'user4', 2);

insert into clients values 
('1234567891', 'client name 1', 'client phone 1', 'client email 1', now(), 'reg address 1', 'rev address 1', 'user1'),
('2345678912', 'client name 2', 'client phone 2', 'client email 2', now(), 'reg address 2', 'rev address 2', 'user2'),
('3456789123', 'client name 3', 'client phone 3', 'client email 3', now(), 'reg address 3', 'rev address 3', 'user3');

insert into cars values 
('123764192341192834912', 1.00, 'car brand 1', 'car color 1', 'car class 1', 'A', 'ai-95', false, 1),
('248597238945703124582', 2.00, 'car brand 2', 'car color 2', 'car class 2', 'M', 'ai-95', false, 2),
('234587923452475245883', 3.00, 'car brand 3', 'car color 3', 'car class 3', 'A', 'ai-95', false, 2);

insert into raw_contracts values
(1, '1 day', 1000.00, '1234567891', '123764192341192834912');

insert into manager_on_contract values
('manager passport 1', 1);


select * from raw_contracts;
select * from archive;


update raw_contracts
set signed = true 
where ID_contract = 1;


select * from archive;
select * from raw_contracts;
select * from manager_on_contract;
*/


INSERT INTO users (username, password, role) VALUES 
('user1', 'password1', 'client'),
('user2', 'password2', 'client'),
('manager', 'manager', 'manager');

insert into parkings values 
(1, 'asfiaslfdgashdfguiapshg'),
(2, 'gflsdjiopsjdfoigaoijbdi');

insert into managers (manager_passport_details, name, phone, expirience, salary, username, ID_parking) values 
('manager passport 1', 'manager name 1', 'manager phone 1', DEFAULT, 10, 'manager', 1);

insert into clients values 
('1234567891', 'client name 1', 'client phone 1', 'client email 1', now(), 'reg address 1', 'rev address 1', 'user1'),
('2345678912', 'client name 2', 'client phone 2', 'client email 2', now(), 'reg address 2', 'rev address 2', 'user2');

insert into cars values 
('123764192341192834912', 1.00, 'car brand 1', 'car color 1', 'car class 1', 'A', 'ai-95', false, 1),
('248597238945703124582', 2.00, 'car brand 2', 'car color 2', 'car class 2', 'M', 'ai-95', false, 2),
('234587923452475245883', 3.00, 'car brand 3', 'car color 3', 'car class 3', 'A', 'ai-95', false, 2);

INSERT 0 3

INSERT 0 2

INSERT 0 1

INSERT 0 2

INSERT 0 3

Total execution time: 00:00:00.029

**Roles**

In [170]:
/*
*/
drop role if exists _admin;
drop role if exists _manager;
drop role if exists _client;

drop role if exists new_user;


-- Create roles


create role _admin createrole inherit nologin;
create role _manager inherit nologin;
create role _client inherit nologin;

------------------------------------------

-- Admin configuration
grant all on all tables in schema public to _admin;


-- Manager configuration
grant select on cars, managers, archive to _manager;
grant insert on archive, accidents to _manager;



-- Client configuration
grant select on cars, clients to _client;
grant insert on raw_contracts to _client;



------------------------------------------



-- Create admin users

DROP ROLE

DROP ROLE

DROP ROLE

DROP ROLE

CREATE ROLE

...

GRANT

GRANT

GRANT

GRANT

Total execution time: 00:00:00.053

**Test**

In [171]:
set role postgres;

call add_user('new_user', 'new_password');
call add_user('user2', 'password2');
call add_user('user3', 'password3');
call add_user('user4', 'password4');

select * from users;

select * from pg_roles;

SET

CALL

CALL

CALL

CALL

(6 row(s) affected)

(25 row(s) affected)

Total execution time: 00:00:00.036

username,password,role
user1,password1,client
user2,password2,client
manager,manager,manager
new_user,new_password,client
user3,password3,client
user4,password4,client


rolname,rolsuper,rolinherit,rolcreaterole,rolcreatedb,rolcanlogin,rolreplication,rolconnlimit,rolpassword,rolvaliduntil,rolbypassrls,rolconfig,oid
pg_database_owner,False,True,False,False,False,False,-1,********,NULL,False,NULL,6171
pg_read_all_data,False,True,False,False,False,False,-1,********,NULL,False,NULL,6181
pg_write_all_data,False,True,False,False,False,False,-1,********,NULL,False,NULL,6182
pg_monitor,False,True,False,False,False,False,-1,********,NULL,False,NULL,3373
pg_read_all_settings,False,True,False,False,False,False,-1,********,NULL,False,NULL,3374
pg_read_all_stats,False,True,False,False,False,False,-1,********,NULL,False,NULL,3375
pg_stat_scan_tables,False,True,False,False,False,False,-1,********,NULL,False,NULL,3377
pg_read_server_files,False,True,False,False,False,False,-1,********,NULL,False,NULL,4569
pg_write_server_files,False,True,False,False,False,False,-1,********,NULL,False,NULL,4570
pg_execute_server_program,False,True,False,False,False,False,-1,********,NULL,False,NULL,4571


**Polices**

In [172]:
set role postgres;
alter table cars enable row level security;


create policy manager_select_cars_policy on cars
    for select to _manager
    using ((select ID_parking from managers where username = current_user) = ID_parking);

create policy client_select_cars_policy on cars 
    for select to _client 
    using (locked = false);


SET

ALTER TABLE

CREATE POLICY

CREATE POLICY

Total execution time: 00:00:00.040

**Test**

In [173]:
set role postgres;

select * from cars;

set role new_user;

select * from cars;

SET

(3 row(s) affected)

SET

(3 row(s) affected)

Total execution time: 00:00:00.008

vin_number,cost_per_hour,car_brand,car_color,car_class,transmission_type,fuel_type,locked,id_parking
123764192341192834912,$1.00,car brand 1,car color 1,car class 1,A,ai-95,False,1
248597238945703124582,$2.00,car brand 2,car color 2,car class 2,M,ai-95,False,2
234587923452475245883,$3.00,car brand 3,car color 3,car class 3,A,ai-95,False,2


vin_number,cost_per_hour,car_brand,car_color,car_class,transmission_type,fuel_type,locked,id_parking
123764192341192834912,$1.00,car brand 1,car color 1,car class 1,A,ai-95,False,1
248597238945703124582,$2.00,car brand 2,car color 2,car class 2,M,ai-95,False,2
234587923452475245883,$3.00,car brand 3,car color 3,car class 3,A,ai-95,False,2
